In [1]:
subs = None
media = 'Jojo Stone Ocean 01.mkv'
altsubs = None
track = None
if subs == None:
    import subprocess
    filename = media[:-3] + 'ass'
    if track == None:
        try:
            cmd = ['C:\Program Files\MKVToolNix\mkvextract', 'tracks', media, '2:' + filename]
            subprocess.run(cmd, check=True)
            f = open(filename)
            f.read()
        except:
            i = 2
            while i<20:
                try:
                    cmd = ['C:\Program Files\MKVToolNix\mkvextract', 'tracks', media, str(i) + ':' + filename]
                    subprocess.run(cmd, check=True)
                    f = open(filename)
                    f.read()
                    print('Track', i, 'has subtitles.')
                    i=20
                except:
                    print('Track', i, 'does not have subtitles.')
                    track = i
                    i+=1
    else:
        cmd = ['C:\Program Files\MKVToolNix\mkvextract', 'tracks', media, str(track) + ':' + filename]
        subprocess.run(cmd, check=True)
else:
    filename = subs

if altsubs != None:
    f = open(altsubs, encoding='utf-8')
    text = f.read()
    text = text.replace('â€¦', '!')
    text = text.replace('Â\xa0', '')
    text = text.replace('\r', '')
    text = text.replace('â™ª', '')
    text = text.replace('â€™', "'")
    text = text.replace('Ê¼', "'")
    altsubsdict = dict()
    altlst = text.split('\n')
    alttext = ""
    for i in range(len(altlst)):
        if '-->' in altlst[i]:
            alttext += altlst[i+1] + " " + altlst[i+2] + " "
            alttxt = (altlst[i+1] + " " + altlst[i+2]).rstrip()
            alttxt = alttxt.replace('\u202a', '')
            alttxt = alttxt.replace('\u202c', '')
            alttxt = alttxt.replace('\u3000', '')
            alttxt = alttxt.replace('&lrm;', '')
            altsubsdict[altlst[i]] = alttxt

def ass2srt(filename):
    import asstosrt
    try:
        ass_file = open(filename, encoding='utf-8')
        text = asstosrt.convert(ass_file)
    except Exception as e:
        text = ass_file.read()
        print(e)
    text = text.replace('â€¦', '!')
    text = text.replace('Â\xa0', '')
    text = text.replace('\r', '')
    text = text.replace('â™ª', '')
    text = text.replace('â€™', "'")
    text = text.replace('Ê¼', "'")
    return text

if filename[-3:] == 'ass':
    text = ass2srt(filename)
elif filename[-3:] == 'srt':
    f = open(filename, encoding='utf-8')
    text = f.read()
elif filename[-3:] == 'txt':
    f = open(filename, encoding='utf-8')
    text = f.read()
    
text = text.replace('â€¦', '!')
text = text.replace('Â\xa0', '')
text = text.replace('\r', '')
text = text.replace('â™ª', '')
text = text.replace('â€™', "'")
text = text.replace('Ê¼', "'")

lst = text.split('\n')
timestamps = list()
ilst = list()
tlst = list()
subs = dict()
subsn = dict()
textf = ""
z=0
for i in range(len(lst)):
    if '-->' in lst[i]:
        textf += lst[i+1] + " " + lst[i+2] + " "
        txt = (lst[i+1] + " " + lst[i+2]).rstrip()
        subs[lst[i]] = txt

In [2]:
jpname = None
endswithS0xE0x = False
newnum = None

if altsubs == None:
    name = media[:-4]
    name = name.replace(' - ', ' ')
    scoremax = 0

    import requests
    from bs4 import BeautifulSoup

    response = requests.get('https://kitsunekko.net/dirlist.php?dir=subtitles%2Fjapanese%2F')
    soup = BeautifulSoup(response.text, 'html.parser')
    strong_tags = soup.find_all('strong')
    
    for tag in strong_tags:
        words = tag.text.split(' ')
        namelst = name.split(' ')
        score = 0
        for i in words:
            for j in namelst:
                if i == j:
                    score += 1
        if score > scoremax:
            scoremax = score
            anime = tag.text
            a_tag = tag.find_parent('a')
            href = a_tag.get('href')
        if jpname != None:
            words = tag.text.split(' ')
            namelst = jpname.split(' ')
            score = 0
            for i in words:
                for j in namelst:
                    if i == j:
                        score += 1
            if score > scoremax:
                scoremax = score
                anime = tag.text
                a_tag = tag.find_parent('a')
                href = a_tag.get('href')
    print(anime, 'detected.')
        
    scoremax = 0
    response = requests.get('https://kitsunekko.net'+href)
    soup = BeautifulSoup(response.text, 'html.parser')
    strong_tags = soup.find_all('strong')
    for tag in strong_tags:
        word = tag.text
        word = word.replace('.srt', '')
        words = word.split(' ')
        namelst = name.split(' ')
        if endswithS0xE0x == True:
            namelst.append(name[-5:-3])
            namelst.append(name[-2:])
        for i in namelst:
            if i.isdigit():
                num = int(i)
        for i in words:
            if i.isdigit():
                newnum = int(i)
        if num == newnum:
            episode = tag.text
            a_tag = tag.find_parent('a')
            href = a_tag.get('href')

    print(episode, 'detected.')
    response = requests.get('https://kitsunekko.net/'+href)

    with open(name+'.srt', 'wb') as f:
        f.write(response.content)
        

    print(name + '.srt downloaded')
    
    altsubs = name + '.srt'
    
    if episode[-3:] == 'srt':
        f = open(altsubs, encoding='utf-8')
        text = f.read()
    else:
        text = ass2srt(altsubs)
        os.remove(name+'.srt')
        with open(name+'.srt', 'w', encoding='utf-8') as f:
            f.write(text)

    text = text.replace('â€¦', '!')
    text = text.replace('Â\xa0', '')
    text = text.replace('\r', '')
    text = text.replace('â™ª', '')
    text = text.replace('â€™', "'")
    text = text.replace('Ê¼', "'")
    text = text.replace('\t', '')
    altsubsdict = dict()
    altlst = text.split('\n')
    alttext = ""

    for i in range(len(altlst)):
        if '-->' in altlst[i]:
            alttext += altlst[i+1] + " " + altlst[i+2] + " "
            alttxt = (altlst[i+1] + " " + altlst[i+2]).rstrip()
            alttxt = alttxt.replace('\u202a', '')
            alttxt = alttxt.replace('\u202c', '')
            alttxt = alttxt.replace('\u3000', '')
            alttxt = alttxt.replace('&lrm;', '')
            altsubsdict[altlst[i]] = alttxt

JoJo no Kimyou na Bouken: Stone Ocean detected.
JoJo no Kimyou na Bouken Stone Ocean [Netflix] 01.srt detected.
Jojo Stone Ocean 01.srt downloaded


In [3]:
from datetime import datetime
deltan = datetime.strptime('00:00:10.000', '%H:%M:%S.%f') - datetime.strptime('00:00:00.000', '%H:%M:%S.%f')
altdict = dict()
for k, v in subs.items():
    time1 = k.split(' --> ')[0]
    for a, b in altsubsdict.items():
        time2 = a.split(' --> ')[0]
        if datetime.strptime(time1.replace(',', '.').lstrip(), '%H:%M:%S.%f') > datetime.strptime(time2.replace(',', '.').rstrip(), '%H:%M:%S.%f'):
            delta = datetime.strptime(time1.replace(',', '.').lstrip(), '%H:%M:%S.%f') - datetime.strptime(time2.replace(',', '.').rstrip(), '%H:%M:%S.%f')
        else:
            delta = datetime.strptime(time2.replace(',', '.').lstrip(), '%H:%M:%S.%f') - datetime.strptime(time1.replace(',', '.').rstrip(), '%H:%M:%S.%f')
        if delta < deltan:
            deltan = delta
            time = k
            text = b
    try:
        textbefore = aldict[time]
        altdict[time] = textbefore + " " + text
        print("Pass")
    except:
        altdict[time] = text
    deltan = datetime.strptime('00:00:10.000', '%H:%M:%S.%f') - datetime.strptime('00:00:00.000', '%H:%M:%S.%f')
    
japsubs = list()

for k, v in altdict.items():
    japsubs.append(v)

In [4]:
import os
import subprocess
from datetime import datetime
from ipywidgets import IntProgress, Label, HBox
from IPython.display import display

progress = 0
punc = ["!", '"', "#", "$", "%", "&", "'", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", ">", "?", "@", "[", "\\", "]", "^", "_", "`", "{", "|", "}", "~", ","]

factor = 0.05

label = Label()
progressbar = IntProgress(value=0, min=0, max=100)
hbox = HBox([progressbar, label])
display(hbox)
    
medialst = list()
audiomedia = list()
imagemedia = list()
mediaonly = list()
audioonly = list()
imageonly = list()
tuplst = list()
total = len(subs)

cwd = os.getcwd()

output_folder = f'{cwd}\\Temp'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for tym, name in subs.items():
    time = tym.split(' --> ')
    fname = name
    for s in punc:
        name = name.replace(s, '')
    output_file = f'{output_folder}\output.mp4'

    file = f'{output_folder}\\' + name + '.mp4'
    audio = f'{output_folder}\\' + name + '.mp3'
    image = f'{output_folder}\\' + name + '.png'
    if os.path.exists(file) & os.path.exists(audio) & os.path.exists(image):
        pass
    else:
        delta = datetime.strptime(time[1].replace(',', '.').lstrip(), '%H:%M:%S.%f') - datetime.strptime(time[0].replace(',', '.').rstrip(), '%H:%M:%S.%f')
        s = datetime.strptime(time[0].replace(',', '.').lstrip(), '%H:%M:%S.%f') - delta * factor
        e = datetime.strptime(time[1].replace(',', '.').lstrip(), '%H:%M:%S.%f') + delta * factor
        itime = s.strftime('%H:%M:%S.%f')
        etime = e.strftime('%H:%M:%S.%f')
        ffmpeg_command = ['ffmpeg', '-y', '-ss', itime, '-to', etime, '-i', media[:-4] + '.mkv', '-c:v', 'copy', '-c:a', 'copy', file]
        subprocess.run(ffmpeg_command)
        ffmpeg_command = ['ffmpeg', '-y', '-i', file, '-vn', '-acodec', 'libmp3lame', audio]
        subprocess.run(ffmpeg_command)
        ffmpeg_command = ['ffmpeg', '-y', '-ss', '0', '-i', file, '-f', 'image2', '-vframes', '1', image]
        subprocess.run(ffmpeg_command)
    medialst.append(file)
    audiomedia.append(audio)
    imagemedia.append(image)
    mediaonly.append(name + '.mp4')
    audioonly.append(name + '.mp3')
    imageonly.append(name + '.png')
    tup = (fname, file)
    tuplst.append(tup)
    progress+=1
    label.value = f'{progress/total*100:.1f}%'
    progressbar.value = progress/total*100

In [5]:
import genanki
import pykakasi
kks = pykakasi.kakasi()
import jisho_api
from jisho_api.kanji import Kanji
import json
import sqlite3

conn = sqlite3.connect('JPcache.db')
cursor = conn.cursor()
cursor.execute('CREATE TABLE IF NOT EXISTS cache (query text, result text)')
conn.close()

def KanjiRequest(query):
    conn = sqlite3.connect('JPCache.db')
    cursor = conn.cursor()
    cursor.execute('SELECT result FROM cache WHERE query=?', (query,))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        response = Kanji.request(query)
        result = response.json()
        cursor.execute('INSERT INTO cache (query, result) VALUES (?, ?)', (query, result))
        conn.commit()
        return result

progress = 0

label = Label()
progressbar = IntProgress(value=0, min=0, max=100)
nlabel = Label()
hbox = HBox([progressbar, label, nlabel])
display(hbox)

punc = ["!", '"', "#", "$", "%&", "'", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", ">", "?", "@", "[", "\\", "]", "^", "_", "`", "{", "|", "}", "~", "･"]
model = genanki.Model(
  1601234269,
  'Model',
  fields=[
    {'name': 'Japanese Text'},
    {'name': 'Image'},
    {'name': 'Answer'},
    {'name': 'Video'},
    {'name': 'Audio'},
    {'name': 'Primary Kanji to Hiragana'},
    {'name': 'Rest Kanji to Hiragana'}
  ],
  templates=[
    {
      'name': 'Card 1',
      'qfmt': '{{Japanese Text}}<br>{{Image}}<br>Audio<br>{{Audio}}<br>Video<br>{{Video}}',              
      'afmt': '{{FrontSide}}<hr id="answer">{{Answer}}<br>{{Primary Kanji to Hiragana}}<br><details><summary>Other Kanji Meanings</summary>{{Rest Kanji to Hiragana}}</details>',
    },
  ],
  css = """
    .card {
     font-family: times;
     font-size: 30px;
     text-align: center;
     color: black;
     background-color: white;
    }

    .card1 { background-color:Lavender }
    
    img {
    height: auto;
    max-width: 100%;
    }
    
    """
)

deck = genanki.Deck(
  205123510,
  media[:-4])

package = genanki.Package(deck)
mediafiles = list()

for i in range(len(tuplst)):
    try:
        extra = ""
        token = kks.convert(japsubs[i])
        japtxt = ""
        for a in token:
            if a['orig'] == a['hira']:
                pass
            elif a['orig'] == a['kana']:
                pass
            elif a['orig'] in punc:
                pass
            else:
                try:
                    kanji = json.loads(KanjiRequest(a['orig']))
                    r = kanji['data']['main_meanings']
                    japtxt += a['orig'] + ' → ' + a['hira'] + '<br>'
                    if len(r) == 1:
                        for s in r:
                            japtxt += a['orig'] + ' → ' + s + '<br>'
                    else:
                        num = 0
                        for s in r:
                            if num == 0:
                                japtxt += a['orig'] + ' → ' + s + '<br>'
                                num = 1
                            else:
                                extra += a['orig'] + ' → ' + s + '<br>'
                except Exception as e:
                    print(e)
                    japtxt += '<br>' + 'ERROR'
        my_note = genanki.Note(
          model = model,
          fields = [japsubs[i], '<img src="{}">'.format(imageonly[i]), tuplst[i][0], '[sound:{}]'.format(mediaonly[i]), '[sound:{}]'.format(audioonly[i]), japtxt, extra])
        deck.add_note(my_note)
        mediafiles.append('{}'.format(medialst[i]))
        mediafiles.append('{}'.format(audiomedia[i]))
        mediafiles.append('{}'.format(imagemedia[i]))
        #print("Card successfully generated.")
    except:
        #print("Card generation failed. Committing seppuku.")
        pass
    progress +=1
    label.value = f'{progress/total*100:.1f}%'
    nlabel.value = f'(Cards Generated: {progress})'
    progressbar.value = progress/total*100
package.media_files = mediafiles
package.write_to_file(media[:-4] + '.apkg')

[Error] No kanji found with name おらァ！.

'NoneType' object has no attribute 'json'


In [6]:
#for i in mediafiles:
    #try:
        #os.remove(i)
    #except:
        #pass

import shutil
shutil.rmtree('Temp')